# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
import copy
import scipy
import pandas
import random
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


from scipy.stats import mode
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [2]:
def gini(x):
    proba = np.bincount(x) / x.shape[0]
#     _, counts = np.unique(x, return_counts=True)
#     proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    proba = np.bincount(x) / x.shape[0]
#     _, counts = np.unique(x, return_counts=True)
#     proba = counts / len(x)
    return -np.sum(proba * np.log2(proba + 1e-16))

def gain(left_y, right_y, criterion):
    N = len(left_y) + len(right_y)
    
    return (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / N

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [7]:
class Node:
    def __init__(self, split_dim, split_value, left=None, right=None):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left, self.right = left, right

class Leaf:
    def __init__(self, labels, n_classes):
        self.prediction = self._predict(labels, n_classes)
        
    def _predict(self, x, n_classes):
        prediction = np.zeros(n_classes)
    
        labels, counts = np.unique(x, return_counts=True)
        prediction[labels] = counts / x.shape[0]
        
        return prediction

class DecisionTree:
    def __init__(self, X, y, n_classes, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        criterions = {'gini': gini, 'entropy': entropy}
        
        self.criterion = criterions[criterion]
        self.max_depth = max_depth if max_depth is not None else np.inf
        self.max_features = np.sqrt(X.shape[1]).astype(np.int) if max_features == "auto" else max_features
        self.min_samples_leaf = min_samples_leaf

        self.train_idx, self.oob_idx = self._bootstrap_train(X.shape[0])

        self.n_classes = n_classes
        self.root = self._fit(X[self.train_idx], y[self.train_idx])


    def _bootstrap_train(self, N):
        sample_idx = np.random.choice(N, N, replace=True)
        
        return sample_idx, np.setdiff1d(np.arange(N), sample_idx)
    
    def _fit(self, X, y, depth=1):
        if depth >= self.max_depth:
            return Leaf(y, self.n_classes)
        
        split_dim, split_value = None, None
        
        # find best split
        best_gain = np.inf
        for feature in np.random.choice(X.shape[1], self.max_features, replace=False):
            split_mask = X[:, feature] < 0.5 # only bool features

            y_left, y_right = y[split_mask], y[~split_mask]
            gain_value = gain(y_left, y_right, self.criterion)

            not_leaf = y_left.shape[0] >= self.min_samples_leaf and y_right.shape[0] >= self.min_samples_leaf

            if gain_value < best_gain and not_leaf:
                split_dim, split_value, best_gain = feature, 0.5, gain_value

        if split_dim is None:
            return Leaf(y, self.n_classes)
    
        split_mask = X[:, split_dim] < split_value
        
        root = Node(split_dim, split_value)
        root.left = self._fit(X[split_mask], y[split_mask], depth + 1)
        root.right = self._fit(X[~split_mask], y[~split_mask], depth + 1)
        
        return root
    
    def _predict_row(self, x, root):
        if isinstance(root, Leaf):
            return root.prediction
        
        if x[root.split_dim] < root.split_value:
            return self._predict_row(x, root.left)
        else:
            return self._predict_row(x, root.right)
        
    def predict_proba(self, X):
        probs = np.zeros((X.shape[0], self.n_classes))

        for i, row in enumerate(X):
            probs[i] = self._predict_row(row, self.root)

        return probs
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [8]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.estimators = None

        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
    
    def fit(self, X, y):        
        n_classes = np.unique(y).shape[0]
        
        self.estimators = []
        for _ in range(self.n_estimators):
            tree = DecisionTree(X, y, n_classes, self.criterion, self.max_depth, 
                                self.min_samples_leaf, self.max_features)
            self.estimators.append(tree)
                           
        return self
    
    def predict(self, X):
        if self.estimators is None:
            raise AttributeError("Fit model first!")
            
        prediction = np.zeros((self.n_estimators, X.shape[0]))
        
        for i, estimator in enumerate(self.estimators):
            prediction[i] = estimator.predict(X)
        
        return scipy.stats.mode(prediction, axis=0)[0].ravel()

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [9]:
def feature_importance(rfc, X, y):
    importance = np.zeros((rfc.n_estimators, X.shape[1]))
    
    err_oob = [(1 - np.mean(tree.predict(X[tree.oob_idx]) == y[tree.oob_idx])) for tree in rfc.estimators]
    
    for feature in range(X.shape[1]):
        X_perm = X.copy()
        X_perm[:, feature] = np.random.permutation(X_perm[:, feature])

        for i, tree in enumerate(rfc.estimators):
            X_oob, y_oob = X_perm[tree.oob_idx], y[tree.oob_idx]
            
            err_oob_i = 1 - np.mean(tree.predict(X_oob) == y_oob)
            importance[i, feature] = err_oob_i - err_oob[i]        
    
    return importance.mean(axis=0)

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [10]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc, X, y))

Accuracy: 1.0
Importance: [-3.41187787e-04  7.26933540e-05  1.45667306e-01  1.42977053e-01
  3.20689228e-01 -5.29974508e-05]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [11]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [12]:
X, y_age, y_sex, features = read_dataset("vk.csv")

y_age_labels, y_age = np.unique(y_age, return_inverse=True)
y_sex_labels, y_sex = np.unique(y_sex, return_inverse=True)

X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [13]:
%%time
rfc = RandomForestClassifier(n_estimators=10, criterion="entropy")
rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))

Accuracy: 0.7099621689785625
CPU times: user 51.8 s, sys: 1.45 s, total: 53.3 s
Wall time: 55.3 s


In [14]:
%%time
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc, X_train, y_age_train), features, 20)):
    print(str(i+1) + ".", name)

Most important features:
1. ovsyanochan
2. 4ch
3. mudakoff
4. rhymes
5. styd.pozor
6. pravdashowtop
7. pustota_diary
8. pixel_stickers
9. rapnewrap
10. bot_maxim
11. iwantyou
12. dayvinchik
13. tumblr_vacuum
14. dzenpub
15. leprum
16. xfilm
17. ohhluul
18. ne1party
19. femalemem
20. bestad
CPU times: user 6min 2s, sys: 3.43 s, total: 6min 5s
Wall time: 6min 26s


#### Пол

In [12]:
%%time
rfc = RandomForestClassifier(n_estimators=10, criterion="entropy")
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))

Accuracy: 0.8575031525851198
CPU times: user 48.5 s, sys: 1.59 s, total: 50.1 s
Wall time: 53 s


In [13]:
%%time
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc, X_train, y_sex_train), features, 20)):
    print(str(i+1) + ".", name)

Most important features:
1. leprum
2. 21jqofa
3. i.kino
4. ne.poverish
5. memeboizz
6. leprazo
7. komment.broo
8. top_screens
9. ru.esquire
10. mash
11. pho
12. rem_shkola
13. webestano
14. ultrapir
15. ftp_memes
16. fucking_humor
17. vandroukiru
18. dzenpub
19. morgenshtern666
20. vulgaarr
CPU times: user 6min 26s, sys: 7.32 s, total: 6min 33s
Wall time: 7min 33s


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [29]:
X, y = synthetic_dataset(1000)

model = CatBoostClassifier(loss_function='MultiClass', verbose=False)
model.fit(X, y)

print("Accuracy:", np.mean(model.predict(X).flatten() == y))
print("Importance:", model.feature_importances_)

Accuracy: 1.0
Importance: [6.46582749e-03 1.28538611e-02 2.78343165e+01 2.78247365e+01
 4.43118985e+01 9.72883594e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [30]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [42]:
model = CatBoostClassifier(loss_function='MultiClass', verbose=False)
model.fit(X_train, y_age_train)

print("Accuracy:", np.mean(model.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, (imp, name) in enumerate(sorted(zip(model.feature_importances_, features), reverse=True)[:10]):
    print(str(i+1) + ".", name, ":", round(imp, 4))

Accuracy: 0.7641866330390921
Most important features:
1. ovsyanochan : 2.9137
2. mudakoff : 2.9035
3. 4ch : 2.2852
4. styd.pozor : 2.2254
5. rhymes : 2.0737
6. dayvinchik : 1.93
7. xfilm : 1.8566
8. leprum : 1.7797
9. rapnewrap : 1.7214
10. fuck_humor : 1.5567


#### Пол

In [43]:
model = CatBoostClassifier(loss_function='MultiClass', verbose=False)
model.fit(X_train, y_sex_train)

print("Accuracy:", np.mean(model.predict(X_test).flatten() == y_sex_test))
print("Most important features:")

for i, (imp, name) in enumerate(sorted(zip(model.feature_importances_, features), reverse=True)[:10]):
    print(str(i+1) + ".", name, ":", round(imp, 4))

Accuracy: 0.8738965952080706
Most important features:
1. 40kg : 4.0741
2. mudakoff : 3.1424
3. modnailru : 2.3034
4. girlmeme : 2.2499
5. rapnewrap : 1.7651
6. 4ch : 1.6958
7. academyofman : 1.6609
8. 9o_6o_9o : 1.6199
9. femalemem : 1.5404
10. i_d_t : 1.5252
